In [ ]:
import os
os.environ['HF_HOME'] = '/media/lobby/KINGSTON/.cache/huggingface'
os.environ['HF_HOME']

In [ ]:

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
# torch_dtype



In [ ]:
# model_id = "openai/whisper-large-v3-turbo"
model_id = "openai/whisper-tiny"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=1,  # batch size for inference - set based on your device
    torch_dtype=torch_dtype,
    device=device,
)


In [ ]:
# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
# result = pipe(sample)

# specify the local audio file
result = pipe("input/audio/jp-short.mp3")


srtFilename = os.path.join("output/transcripts", f"jp-short.srt")

with open(srtFilename, 'a', encoding='utf-8') as srtFile:
    srtFile.write(result["text"])

srtFilename